# Codigo propio

In [1]:
import lib.custom_metrics as metrics
import lib.evaluation as ev
import lib.plotting as plot
import lib.models as models
import lib.dataset as dt

import matplotlib.pyplot as plt
import imgaug.augmenters as iaa
import tensorflow as tf
import pandas as pd
import numpy as np
import datetime
import os

from importlib import reload

In [2]:
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  # Restrict TensorFlow to only use the first GPU
  try:
    tf.config.experimental.set_visible_devices(gpus[0], 'GPU')
    logical_gpus = tf.config.experimental.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPU")
  except RuntimeError as e:
    # Visible devices must be set before GPUs have been initialized
    print(e)

1 Physical GPUs, 1 Logical GPU


In [5]:
DR_LEVELS_PER_CLASS = [[0], [1,2,3,4]]

IMAGE_SIZE = (540, 540, 3)

# Specify dataset files
TRAIN_FILE = 'DATASET-VALIDATION-10.csv'
VALIDATION_FILE = 'DATASET-TEST-10.csv'

TRAINING_BATCH_SIZE = 8
TRAINING_DATA_AUG = False
TRAINING_BALANCED = True #-----
TRAINING_SHUFFLE = False
TRAINING_PREFETCH = None
TRAINING_TAKE_SIZE = None

VALIDATION_BATCH_SIZE = 8
VALIDATION_DATA_AUG = False
VALIDATION_BALANCED = True #-----
VALIDATION_SHUFFLE = False
VALIDATION_PREFETCH = None
VALIDATION_TAKE_SIZE = None

In [6]:
reload(dt)

'''
def create_dataset_new(csv_file, 
                        list_list_classes,
                        balanced=False,
                        apply_data_augmentation=False,
                        batch_size=1, 
                        prefetch_buffer=None, 
                        shuffle=False,
                        size=None):
'''

train_dataset, y_true_train = dt.create_dataset_new(TRAIN_FILE, 
                                                    DR_LEVELS_PER_CLASS, 
                                                    balanced=TRAINING_BALANCED, 
                                                    apply_data_augmentation=TRAINING_DATA_AUG, 
                                                    batch_size=TRAINING_BATCH_SIZE, 
                                                    prefetch_buffer=TRAINING_PREFETCH, 
                                                    shuffle=TRAINING_SHUFFLE,
                                                    size=TRAINING_TAKE_SIZE)

val_dataset, y_true_val = dt.create_dataset_new(VALIDATION_FILE, 
                                                DR_LEVELS_PER_CLASS, 
                                                balanced=VALIDATION_BALANCED,
                                                apply_data_augmentation=VALIDATION_DATA_AUG,
                                                batch_size=VALIDATION_BATCH_SIZE,
                                                prefetch_buffer=VALIDATION_PREFETCH, 
                                                shuffle=VALIDATION_SHUFFLE, 
                                                size=VALIDATION_TAKE_SIZE)

This balanced dataset will have 3598 images, having 1799 images per class
This balanced dataset will have 3758 images, having 1879 images per class


In [7]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32,(5,5), strides=2, input_shape=IMAGE_SIZE),
    tf.keras.layers.ReLU(), # 268, 268
    tf.keras.layers.Conv2D(32,(7,7), strides=5),
    tf.keras.layers.ReLU(), # 53, 53
    tf.keras.layers.Conv2D(32,(3,3), strides=2),
    tf.keras.layers.ReLU(), # 26, 26
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(48, activation='relu'),
    tf.keras.layers.Dropout(0.25),
    tf.keras.layers.Dense(len(DR_LEVELS_PER_CLASS), activation='softmax')
])

model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [8]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 268, 268, 32)      2432      
_________________________________________________________________
re_lu (ReLU)                 (None, 268, 268, 32)      0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 53, 53, 32)        50208     
_________________________________________________________________
re_lu_1 (ReLU)               (None, 53, 53, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 26, 26, 32)        9248      
_________________________________________________________________
re_lu_2 (ReLU)               (None, 26, 26, 32)        0         
_________________________________________________________________
flatten (Flatten)            (None, 21632)             0

In [9]:
# Load the TensorBoard notebook extension
%load_ext tensorboard

cbacks = [tf.keras.callbacks.TensorBoard('logs/EYEPACS_Seq_Modelo_propio (2 datasets bal)', histogram_freq=1, write_graph=False)]

In [10]:
# See Tensorboard
%tensorboard --logdir logs

Reusing TensorBoard on port 6006 (pid 2790), started 2 days, 6:56:57 ago. (Use '!kill 2790' to kill it.)

In [11]:
model.fit(train_dataset, epochs=200, validation_data=val_dataset, verbose=1, callbacks=cbacks)

Epoch 1/200
  1/450 [..............................] - ETA: 0s - loss: 0.7686 - accuracy: 0.0000e+00WARNING:tensorflow:From /home/alumno/miguel_herrera/lib/python3.8/site-packages/tensorflow/python/ops/summary_ops_v2.py:1277: stop (from tensorflow.python.eager.profiler) is deprecated and will be removed after 2020-07-01.
Instructions for updating:
use `tf.profiler.experimental.stop` instead.
450/450 [==============================] - 15s 34ms/step - loss: 0.7262 - accuracy: 0.9864 - val_loss: 239.2941 - val_accuracy: 0.5000
Epoch 2/200
450/450 [==============================] - 15s 33ms/step - loss: 3.5801 - accuracy: 0.9402 - val_loss: 0.9848 - val_accuracy: 0.5000
Epoch 3/200
450/450 [==============================] - 15s 33ms/step - loss: 0.8127 - accuracy: 0.4944 - val_loss: 0.7507 - val_accuracy: 0.5000
Epoch 4/200
450/450 [==============================] - 15s 33ms/step - loss: 0.7283 - accuracy: 0.4630 - val_loss: 0.7215 - val_accuracy: 0.5000
Epoch 5/200
450/450 [==============